# QA/QC of eye-tracking data

This notebook visualizes ET data for the purpose of QC'ing the BIDS conversion.

In [ ]:
%matplotlib inline
from pathlib import Path
import json
import ppjson
from importlib import reload  # For debugging purposes

import numpy as np
import pandas as pd

import eyetrackingrun as et
from matplotlib import pyplot as plt
import plot

from IPython.display import HTML
from matplotlib import animation
import matplotlib.image as mpimg

In the schedule.tsv file, we've listed the EDF files we created and their associated sessions. Now, let's check out what's in the file.

In [ ]:
edf_lookup = pd.read_csv("schedule.tsv", sep="\t", na_values="n/a")
edf_lookup

As an illustrative example, we'll handle the data from session 4. Replace `DATA_PATH` with your data's path.

In [ ]:
DATA_PATH = Path("/data/datasets/hcph-pilot-sourcedata/recordings/psychopy/")
session = 4
et_session = edf_lookup[edf_lookup.session == session]

# Eye-tracking during the diffusion weighted imaging (DWI) run

Let's begin with the DWI run of the session selected above.
We first create a new `EyeTrackingRun` object, encapsulating eye-tracking information in BIDS-like format.
The corresponding *Psychopy* experiment sends the message `hello fixation` and `bye fixation` when the DWI run starts and stops, respectively.

In [ ]:
et_dwi = et.EyeTrackingRun.from_edf(
    DATA_PATH / et_session.fixation_edf.values[0],
    message_first_trigger="hello",
    message_last_trigger="bye",
)

The `et_dwi` object now has three relevant members: metadata, events and recording.

In [ ]:
print(
    json.dumps(et_dwi.metadata, sort_keys=True, indent=2, cls=ppjson.CompactJSONEncoder)
)

In [ ]:
et_dwi.events

In [ ]:
et_dwi.recording

In [ ]:
data = et_dwi.recording[["eye1_x_coordinate", "eye1_y_coordinate"]]

data[data.eye1_x_coordinate.notna() & data.eye1_y_coordinate.notna()]

### Plotting some data

Let's first generate a time axis `t_axis` in seconds.
To do so, we read the "timestamp" column of the dataframe, and divide by the sampling frequency (stored within the metadata).
In order to make it more "BIDS-y", we also set the start of the run at zero by applying the start time metadata.

In [ ]:
t_axis = (
    et_dwi.recording.timestamp.values - et_dwi.metadata["StartTime"]
) / et_dwi.metadata["SamplingFrequency"]

Once we have the sampling axis, we can look at a basic measurement: pupil area in arbitrary units (as it comes from the EyeLink tracker).
With our conversion into BIDS, pupil area (column `pa_right` of the EDF file) is mapped to the `eye1_pupil_size` column of the `_eyetrack.tsv.gz` file.
We first create a figure with landscape ratio to better get a sense of the data along time.

In [ ]:
fig = plt.figure(figsize=(16, 2))
plt.plot(
    t_axis,
    et_dwi.recording["eye1_pupil_size"].values,
)

plt.xlabel("time [s]")
plt.ylabel("pupil area [a.u.]")

We can zoom in into the early moments of the run:

In [ ]:
fig = plt.figure(figsize=(16, 2))
plt.plot(
    t_axis,
    et_dwi.recording["eye1_pupil_size"].values,
)

plt.xlabel("time [s]")
plt.ylabel("pupil area [a.u.]")
plt.xlim((-1.0, 10.0))

Next, we look at eye blinks.
The two discontinuities at almost seconds 8 and 9 of the pupil area plot could be related to blinks.

In [ ]:
fig = plt.figure(figsize=(16, 2))
plt.plot(
    t_axis,
    et_dwi.recording["eye1_blink"].values,
)

plt.xlabel("time [s]")
plt.ylabel("eyes closed")

Looks like the eyes (or at least the right eye, which was tracked) were closed during most of the run.
Let's now plot together the first ten seconds of pupil area AND the blinks binary signal.
Indeed, the pupil area drops when blinks are happening.

In [ ]:
fig = plt.figure(figsize=(16, 2))

plt.plot(
    t_axis,
    et_dwi.recording["eye1_pupil_size"].values,
)

plt.plot(
    t_axis,
    et_dwi.recording["eye1_blink"].values * 10000,
)

plt.xlabel("time [s]")
plt.ylabel("pupil area [a.u.]")
plt.xlim((-0.1, 10.0))

We can clean up the pupil area time series by removing data while the eye was closed.
The signal seems to display less artifacts.

In [ ]:
pupil_area = et_dwi.recording["eye1_pupil_size"].values
pupil_area[et_dwi.recording["eye1_blink"] > 0] = np.nan

fig = plt.figure(figsize=(16, 2))

plt.plot(
    t_axis,
    et_dwi.recording["eye1_pupil_size"].values,
)

plt.xlabel("time [s]")
plt.ylabel("pupil area [a.u.]")

We can also see the map of time spent on areas of the screen:

In [ ]:
size = (
    et_dwi.metadata["ScreenAOIDefinition"][1][1],
    et_dwi.metadata["ScreenAOIDefinition"][1][3],
)
data = et_dwi.recording[["eye1_x_coordinate", "eye1_y_coordinate"]]
data = data[et_dwi.recording.eye1_blink < 1]
plot.plot_heatmap_coordinate(data, density=False, screen_size=size)

At the start and end of the DWI run, we presented two central fixation points during 60s, serving as reference markers for the analysis of potential gaze drift (i.e., shifts in gaze coordinates). Let's compare the density maps during these two fixation points to assess the drift during the DWI.

First, we extract the data corresponding to the two fixation points:

In [ ]:
fixation_duration = 60

start_first_fixation = et_dwi.metadata["StartTime"]
stop_first_fixation = start_first_fixation + (
    fixation_duration * et_dwi.metadata["SamplingFrequency"]
)
stop_second_fixation = et_dwi.metadata["StopTime"]
start_second_fixation = stop_second_fixation - (
    fixation_duration * et_dwi.metadata["SamplingFrequency"]
)

data_first_fixation = et_dwi.recording[
    (et_dwi.recording["timestamp"] >= start_first_fixation)
    & (et_dwi.recording["timestamp"] <= stop_first_fixation)
]

data_second_fixation = et_dwi.recording[
    (et_dwi.recording["timestamp"] >= start_second_fixation)
    & (et_dwi.recording["timestamp"] <= stop_second_fixation)
]

We can now create an animation that displays the heatmap of the coordinates during those two fixation points.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))


def update(frame):
    if frame == 0:
        return plot.plot_heatmap_coordinate(
            data_first_fixation, ax=ax, density=False, screen_size=size
        )
    else:
        return plot.plot_heatmap_coordinate(
            data_second_fixation, ax=ax, density=False, screen_size=size
        )


num_frames = 2
anim = animation.FuncAnimation(
    fig, update, frames=num_frames, interval=2000, repeat=True, blit=False
)

plt.close('all')
plt.clf()
HTML(anim.to_html5_video())


It's interesting to observe that the gaze during the second fixation wasn't centered on the screen, which is unexpected. This happens because the participant's eye was mostly closed during that period. To verify this, let's plot the blink data for the last fixation.

In [ ]:
fig = plt.figure(figsize=(16, 2))
plt.plot(
    (data_second_fixation["timestamp"].values - et_dwi.metadata["StartTime"])
    / et_dwi.metadata["SamplingFrequency"],
    data_second_fixation["eye1_blink"].values,
)

plt.xlabel("time [s]")
plt.ylabel("eyes closed")

# Quality Control task (qct)

Now, let's repeat the process for the quality control task. We'll encapsulate the session data and showcase the same visualizations.

In [ ]:
reload(et)
et_qct = et.EyeTrackingRun.from_edf(
    DATA_PATH / et_session.qct_edf.values[0],
    message_first_trigger='hello qct',
    message_last_trigger='bye qct',
)
t_axis = (et_qct.recording.timestamp.values - et_qct.metadata["StartTime"]) / et_qct.metadata["SamplingFrequency"]

First, let's look at the pupil size after filtering out blinks:

In [ ]:
fig = plt.figure(figsize=(16, 2));
plt.plot(
    t_axis[et_qct.recording.eye1_blink == 0],
    et_qct.recording.loc[et_qct.recording.eye1_blink == 0, "eye1_pupil_size"].values,
)

plt.xlabel("time [s]");
plt.ylabel("pupil area [a.u.]");

The duration of eyeblinks provides insights into the participant's wakefulness during the task. Let's visualize the eyeblink duration specifically for the QCT. Here the plot looks very different compared to the DWI run. The short blink durations in the QCT plot suggest that the participant likely kept their tracked eye open throughout the session.

In [ ]:
fig = plt.figure(figsize=(16, 2))
plt.plot(
    t_axis,
    et_qct.recording["eye1_blink"].values,
)

plt.xlabel("time [s]")
plt.ylabel("eyes closed");

The QCT comprises four randomly ordered tasks: visual gratings, fingertapping with hand instructions, cognitive gaze movement, and blank trials with a central fixation point. Messages are sent by the Psychopy experiment at the beginning and end of each subtask, and these are stored in the 'LoggedMessages' field of et_qct.metadata. Let's examine these messages:

In [ ]:
logged_messages=et_qct.metadata['LoggedMessages']
logged_messages

To visualize the data of each subtask separately, let's construct a dictionary. Each entry will encapsulate a dataframe with data from one of these subtasks.

In [ ]:
subtask_dataframes = {}
n_task = 0
for i in range(len(logged_messages)):
    entry = logged_messages[i]
    if "start" in entry[1]:
        start_time = entry[0]
        subtask_type = entry[1].split()[1]
        stop_time = None

        for j in range(i + 1, len(logged_messages)):

            stop_entry = logged_messages[j]
            if "stop" in stop_entry[1] and (subtask_type in stop_entry[1]):
                stop_time = stop_entry[0]
                n_task = n_task + 1
                break

        if stop_time is not None:

            subtask_data = et_qct.recording[
                (et_qct.recording["timestamp"] >= start_time)
                & (et_qct.recording["timestamp"] <= stop_time)
            ]

            df_name = f"df_{subtask_type}_{n_task}"
            subtask_dataframes[df_name] = subtask_data


Now we can plot the heatmap of the gaze coordinates for each of the subtasks.

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3))

size = (800, 600)

def update(frame):
    ax.clear()
    dataframe = list(subtask_dataframes.values())[frame]

    if dataframe is not None:
        plot.plot_heatmap_coordinate(
            dataframe[["eye1_x_coordinate", "eye1_y_coordinate"]],
            ax=ax,
            density=False,
            screen_size=size,
        )

        subtask_name = (
            list(subtask_dataframes.keys())[frame].split("_")[1].capitalize()
        )
        ax.set_title(f"Subtask: {subtask_name}")


num_frames = len(subtask_dataframes)

anim = animation.FuncAnimation(
    fig, update, frames=num_frames, interval=1000, repeat=True, blit=False
)
plt.close("all")
plt.clf()
HTML(anim.to_html5_video())


During cognitive (cog) events, a noticeable bimodal distribution in gaze coordinates emerges, reflecting the participant's delayed reaction to stimuli. The initial portion of the gaze distribution persists on the coordinates of the preceding stimuli, while the subsequent part aligns with the coordinates of the currently displayed stimulus.

# rest

Now, let's do the same for the resting state. We'll encapsulate the session data and show similar visualizations.

In [ ]:
reload(et)
et_rest = et.EyeTrackingRun.from_edf(
    DATA_PATH / et_session.rest_edf.values[0],
    message_first_trigger="start movie",
    message_last_trigger="Bye rs",
)
t_axis = (
    et_rest.recording.timestamp.values - et_rest.metadata["StartTime"]
) / et_rest.metadata["SamplingFrequency"]


Let's first have a look at the blinks duration.

In [ ]:
fig = plt.figure(figsize=(16, 2))
plt.plot(
    t_axis,
    et_rest.recording["eye1_blink"].values,
)

plt.xlabel("time [s]")
plt.ylabel("eyes closed");

The resting state task comprises three events: initial and final fixation points for drift estimation and a 20-minute movie. Let's start by analyzing the data recorded during the movie. At the beginning and end of the movie, the Psychopy experiment sends the messages'start movie' and 'end movie'. The start of the movie correspond to the first trigger sent by the scanner. First, let's extract the data corresponding to the movie.

In [ ]:
start_movie_timestamp = et_rest.metadata["StartTime"]
stop_movie_timestamp = [
    entry[0] for entry in et_rest.metadata["LoggedMessages"] if "end movie" in entry[1]
][0]
data_movie = et_rest.recording[
    (et_rest.recording["timestamp"] >= start_movie_timestamp)
    & (et_rest.recording["timestamp"] <= stop_movie_timestamp)
]

We can now plot the heatmap of the gaze coordinates during the movie.

In [ ]:
plot.plot_heatmap_coordinate(
    data_movie[["eye1_x_coordinate", "eye1_y_coordinate"]],
    density=False,
    screen_size=size,
    background_image="mundaka-image.png",
)


This provides a general overview of the most focal area for the gaze. Now, for a dynamic view of how the gaze moved during the movie, we can run the next cell. It will generate an animated plot illustrating the trajectory of the gaze.

In [ ]:
plt.ioff()
def update(frame, timestamps_per_frame, background_image):
    start_idx = frame * timestamps_per_frame
    end_idx = (frame + 1) * timestamps_per_frame
    if end_idx > len(data_movie):
        end_idx = len(data_movie)
    subset_data = data_movie.iloc[start_idx:end_idx]
    time = int(
        (data_movie.iloc[start_idx]["timestamp"] - et_rest.metadata["StartTime"])
        / et_rest.metadata["SamplingFrequency"]
    )
    avg_x = subset_data["eye1_x_coordinate"].mean()
    avg_y = subset_data["eye1_y_coordinate"].mean()

    ax.clear()
    ax.scatter(avg_x, avg_y, color="red", marker="o", label="Average Gaze Position")
    ax.legend()

    extent = [
        et_rest.metadata["ScreenAOIDefinition"][1][0],
        et_rest.metadata["ScreenAOIDefinition"][1][1],
        et_rest.metadata["ScreenAOIDefinition"][1][3],
        et_rest.metadata["ScreenAOIDefinition"][1][2],
    ]
    background = mpimg.imread(background_image)
    ax.imshow(background, zorder=0, extent=extent, alpha=0.7)
    ax.set_title(f"Time {time} s")
    ax.set_xlim(extent[0], extent[1])
    ax.set_ylim(extent[3], extent[2])
    ax.invert_yaxis()
    plt.xticks([], [])
    plt.yticks([], [])


fig, ax = plt.subplots(figsize=(8, 6))


timestamps_per_frame = 4000
background_image = "mundaka-image.png"
anim = animation.FuncAnimation(
    fig,
    update,
    frames=len(data_movie) // timestamps_per_frame,
    repeat=False,
    blit=False,
    fargs=(timestamps_per_frame, background_image),
)
HTML(anim.to_html5_video())


Now, let's use the two fixation points to get an idea of the drift during the resting state run, similar to the approach used for the DWI run.

In [ ]:
start_first_fixation = [
    entry[0]
    for entry in et_rest.metadata["LoggedMessages"]
    if "start fixation" in entry[1]
][0]
stop_first_fixation = [
    entry[0]
    for entry in et_rest.metadata["LoggedMessages"]
    if "end fixation" in entry[1]
][0]
stop_second_fixation = [
    entry[0]
    for entry in et_rest.metadata["LoggedMessages"]
    if "end fixation" in entry[1]
][1]
start_second_fixation = [
    entry[0]
    for entry in et_rest.metadata["LoggedMessages"]
    if "start fixation" in entry[1]
][1]

data_first_fixation = et_rest.recording[
    (et_rest.recording["timestamp"] >= start_first_fixation)
    & (et_rest.recording["timestamp"] <= stop_first_fixation)
]
data_second_fixation = et_rest.recording[
    (et_rest.recording["timestamp"] >= start_second_fixation)
    & (et_rest.recording["timestamp"] <= stop_second_fixation)
]


In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))


def update(frame):
    if frame == 0:
        return plot.plot_heatmap_coordinate(
            data_first_fixation, ax=ax, density=False, screen_size=size
        )
    else:
        return plot.plot_heatmap_coordinate(
            data_second_fixation, ax=ax, density=False, screen_size=size
        )


num_frames = 2
anim = animation.FuncAnimation(
    fig, update, frames=num_frames, interval=2000, repeat=False, blit=False
)
plt.close("all")
plt.clf()

HTML(anim.to_html5_video())


# bht

Lastly, let's explore the breath-holding task data. To start, we'll look at the changes in pupil size throughout the task.

In [ ]:
reload(et)
et_bht = et.EyeTrackingRun.from_edf(
    DATA_PATH / et_session.bht_edf.values[0],
    message_first_trigger="hello bht",
    message_last_trigger="Bye bht",
)
t_axis = (
    et_bht.recording.timestamp.values - et_bht.metadata["StartTime"]
) / et_bht.metadata["SamplingFrequency"]


In [ ]:
plt.close("all")
plt.clf()
plt.ion()
fig = plt.figure(figsize=(16, 2));
plt.plot(
    t_axis[et_bht.recording.eye1_blink == 0],
    et_bht.recording.loc[et_bht.recording.eye1_blink == 0, "eye1_pupil_size"].values,
)

plt.xlabel("time [s]");
plt.ylabel("pupil area [a.u.]");

In [ ]:
events_to_extract = [
    "start mock block",
    "stop mock block",
    "start bh block",
    "stop bh block",
]

timestamps = {event.replace(" ", "_"): None for event in events_to_extract}

for timestamp, event in et_bht.metadata["LoggedMessages"]:
    if event in events_to_extract:
        timestamps[event.replace(" ", "_")] = int(timestamp)


The breath-holding task included a mock block where participants didn't follow stimuli and five real blocks where they were instructed to breathe in, out, and hold based on the displayed rectangle colors. In the real blocks, participants had a 10-second break after each breath hold, with the screen turning entirely black. The pupil size plot shows an increase during these breaks, notably more pronounced in the second, third, and fourth blocks. To make this clearer, let's mark the break timings directly on the graph:

In [ ]:
sampling_frequency = et_bht.metadata["SamplingFrequency"]
timestamps_in_seconds = {
    event: (timestamp - et_bht.metadata["StartTime"]) / sampling_frequency
    for event, timestamp in timestamps.items()
}

fig = plt.figure(figsize=(16, 4))
plt.plot(
    t_axis[et_bht.recording.eye1_blink == 0],
    et_bht.recording.loc[et_bht.recording.eye1_blink == 0, "eye1_pupil_size"].values,
)

plt.xlabel("time [s]")
plt.ylabel("pupil area [a.u.]")


for event, timestamp in timestamps_in_seconds.items():
    plt.axvline(
        x=timestamp, color="red", linestyle="--", label=f"{event} ({timestamp:.2f} s)"
    )
plt.axvline(
    x=timestamps_in_seconds["start_bh_block"] + 40,
    color="violet",
    linestyle="--",
    label="break",
)
for n in np.arange(1, 5):
    plt.axvline(
        x=timestamps_in_seconds["start_bh_block"] + 40 * (n + 1) + 10 * n,
        color="violet",
        linestyle="--",
    )

plt.legend()


Another interesting observation is a sudden dip in pupil size around 67 seconds from the beginning. This dip coincides with a point where the eye tracking failed, causing a divergence in the y-coordinate. Let's explore this further by visualizing the pupil size and y-coordinate simultaneously around this moment.

In [ ]:
fig, ax1 = plt.subplots(figsize=(16, 4))

ax1.plot(
    t_axis[et_bht.recording.eye1_blink == 0],
    et_bht.recording.loc[et_bht.recording.eye1_blink == 0, "eye1_pupil_size"].values,
    'blue',
    label="Pupil Size"
)
ax1.set_xlabel("time [s]")
ax1.set_ylabel("Pupil Size [a.u.]", color='blue')


ax2 = ax1.twinx()
ax2.plot(
    t_axis,
    et_bht.recording["eye1_y_coordinate"].values,
    'orange', 
    label="Y gaze coordinate"
)
ax2.set_ylabel("Y Coordinate", color='orange')
ax2.tick_params(axis='y', labelcolor='orange')


plt.xlim(67, 68.5);

Now, mirroring our approach for the previous tasks, let's closely examine the eyeblink duration throughout this session to ensure the participant stayed alert.

In [ ]:
fig = plt.figure(figsize=(16, 2))
plt.plot(
    t_axis,
    et_bht.recording["eye1_blink"].values,
)


plt.xlabel("time [s]")
plt.ylabel("eyes closed");

Finally, let's check out the gaze coordinates density map during the task. Since the rectangles were centered on the screen, we expect the gaze to be mostly concentrated here.

In [ ]:
plot.plot_heatmap_coordinate(
    et_bht.recording[["eye1_x_coordinate", "eye1_y_coordinate"]],
    density=False,
    screen_size=size,
)